In [1]:
import pandas as pd
import numpy as np
import json
from geopy.distance import geodesic
import requests
from bs4 import BeautifulSoup
import xgboost

In [2]:
search_query = '19 Cantonment Close'
response = requests.get('https://developers.onemap.sg/commonapi/search?searchVal={}&returnGeom=Y&getAddrDetails=Y&pageNum=1'.format(search_query))
response.status_code

200

In [3]:
json.loads(response.text)

{'found': 1,
 'totalNumPages': 1,
 'pageNum': 1,
 'results': [{'SEARCHVAL': 'CANTONMENT TOWERS',
   'BLK_NO': '19',
   'ROAD_NAME': 'CANTONMENT CLOSE',
   'BUILDING': 'CANTONMENT TOWERS',
   'ADDRESS': '19 CANTONMENT CLOSE CANTONMENT TOWERS SINGAPORE 080019',
   'POSTAL': '080019',
   'X': '28632.6680290076',
   'Y': '28485.091147231',
   'LATITUDE': '1.27388354188483',
   'LONGITUDE': '103.839003328569',
   'LONGTITUDE': '103.839003328569'}]}

In [4]:
latitude = float(json.loads(response.text)['results'][0]['LATITUDE'])
longitude = float(json.loads(response.text)['results'][0]['LONGITUDE'])

In [5]:
#Check retrieved address
retrieved_add = json.loads(response.text)['results'][0]['ADDRESS']
retrieved_add

'19 CANTONMENT CLOSE CANTONMENT TOWERS SINGAPORE 080019'

In [6]:
#Check retrieved latlong
latitude, longitude

(1.27388354188483, 103.839003328569)

In [7]:
mrt_latlong = pd.read_csv('/mrt_full.csv',index_col=0)

In [8]:
mrt_latlong.head()

,line,station_name,year_opened,is_interchange,Latitude,Longitude
0,DT,BUKIT PANJANG MRT STATION SINGAPORE,2015-12,0,1.378626,103.761431
1,DT,CASHEW MRT STATION SINGAPORE,2015-12,0,1.369976,103.764579
2,DT,HILLVIEW MRT STATION SINGAPORE,2015-12,0,1.363038,103.767413
3,DT,BEAUTY WORLD MRT STATION SINGAPORE,2015-12,0,1.341553,103.775747
4,DT,KING ALBERT PARK MRT STATION SINGAPORE,2015-12,0,1.335688,103.783193


In [9]:
flat=(latitude,longitude)

In [10]:
mrt_in_range = mrt_latlong[(abs(mrt_latlong['Latitude']-latitude)<0.036) & (abs(mrt_latlong['Longitude']-longitude)<0.036)]

In [11]:
#Retrieve nearest MRT and the distance
nearest_mrt = None
mrt_dist = 5

for index in range(len(mrt_in_range)):
    mrt_lat=mrt_in_range.iloc[index,4]
    mrt_long=mrt_in_range.iloc[index,5]
    mrt=(mrt_lat,mrt_long)
    
    distance = geodesic(flat,mrt).km
    if distance < mrt_dist:
        mrt_dist = distance
        nearest_mrt = mrt_in_range.iloc[index,1]

In [12]:
#Check the nearest mrt
nearest_mrt

'OUTRAM PARK  MRT STATION SINGAPORE'

In [13]:
#Check the distance
mrt_dist

0.7051112707006479

In [14]:
#Load in latlong for other amenities
sch_latlong = pd.read_csv('./sch_latlong_corrected.csv',index_col=0)
hawker_latlong = pd.read_csv('./hawker_latlong.csv',index_col=0)
sm_latlong = pd.read_csv('./sm_latlong.csv',index_col=0)
sap_sch_latlong = pd.read_csv('./sap_sch_latlong.csv',index_col=0)
autonomous_ind_latlong = pd.read_csv('./autonomous_ind_latlong.csv',index_col=0)
gifted_ind_latlong = pd.read_csv('./gifted_ind_latlong.csv',index_col=0)
ip_ind_latlong = pd.read_csv('./ip_ind_latlong.csv',index_col=0)

In [15]:
#Create a list of MRT stations within 4km (furthest MRT was 2KM)
amenity_in_range = sm_latlong[(abs(sm_latlong[sm_latlong.columns[1]]-latitude)<0.018) & (abs(sm_latlong[sm_latlong.columns[2]]-longitude)<0.018)]

In [16]:
amenity_in_range

,supermarket_name,supermarket_latitude,supermarket_longitude
40,PAN PACIFIC INTERNATIONAL HOLDINGS PTE. LTD.,1.274588,103.843471
52,FOODIE MARKET PLACE PTE. LTD.,1.283451,103.835371
72,WAN SIEW YOONG,1.281801,103.842908
75,HAO MART PTE. LTD.,1.277849,103.830811
92,MEIDI-YA SINGAPORE CO (PTE) LTD,1.291515,103.844586
107,COLD STORAGE SINGAPORE (1983) PTE LTD,1.277372,103.852845
147,COLD STORAGE SINGAPORE (1983) PTE LTD,1.263896,103.821589
176,NTUC FAIRPRICE CO-OPERATIVE LTD,1.286463,103.826935
179,NTUC FAIRPRICE CO-OPERATIVE LTD,1.289778,103.826794
180,NTUC FAIRPRICE CO-OPERATIVE LTD,1.276241,103.843006


In [17]:
#Create dictionary to store search results
amenities_dict={'num_sch':0,'num_hawkers':0,'num_sm':0,'num_sap':0,'num_autonomous':0,'num_gifted':0,'num_ip':0}

#Create a list to iterate trhough, each entry in the format of [amenity key,amneity DataFrame name,distance to check]
amenities = [['num_sch',sch_latlong,2],['num_hawkers',hawker_latlong,1],
             ['num_sm',sm_latlong,1],['num_sap',sap_sch_latlong,2],
             ['num_autonomous',autonomous_ind_latlong,2],['num_gifted',gifted_ind_latlong,2],['num_ip',ip_ind_latlong,2]]

#Retrieve the number of amenities within given distance
for amenity in amenities:
        #Subset the DataFrame to a square of side 2*distance
        amenity_in_range = amenity[1][(abs(amenity[1][amenity[1].columns[1]]-latitude)<0.009*amenity[2])
                                        & (abs(amenity[1][amenity[1].columns[2]]-longitude)<0.009*amenity[2])]
        count = 0
        
        #Check within the subset for amenities within the given distance
        for index in range(len(amenity_in_range)):
            if geodesic(flat,(amenity_in_range.iloc[index,1],amenity_in_range.iloc[index,2])).km < amenity[2]:
                count+=1
        
        #Save the count in the results dictionary
        amenities_dict[amenity[0]]=count

#print to check
print(amenities_dict['num_sch'])
print(amenities_dict['num_hawkers'])
print(amenities_dict['num_sm'])
print(amenities_dict['num_sap'])
print(amenities_dict['num_autonomous'])
print(amenities_dict['num_gifted'])
print(amenities_dict['num_ip'])

6
2
8
0
0
0
0


In [18]:
sch_latlong.head()

,school_name,school_latitude,school_longitude
0,NATIONAL JUNIOR COLLEGE,1.330293,103.804111
1,TEMASEK JUNIOR COLLEGE,1.318493,103.935567
2,HWA CHONG INSTITUTION,1.325889,103.800747
3,BUKIT PANJANG PRIMARY SCHOOL,1.373622,103.769375
4,HAIG GIRLS' SCHOOL,1.382108,103.738613


In [19]:
# for amenity in amenities:
#     amenity_in_range = amenity[1][(abs(amenity[1][amenity[1].columns[1]]-latitude)<0.018) 
#                                   & (abs(amenity[1][amenity[1].columns[2]]-longitude)<0.018)]
    
#     for index in range(len(amenity_in_range)):
#         if geodesic(flat,(amenity_in_range.iloc[index,1],amenity_in_range.iloc[index,2])).km < 2:
#             amenities_dict[amenity[0]]+=1

In [20]:
hdb_transport = pd.read_csv('./hdb_ll_mrt_cp.csv',index_col=0)

In [21]:
hdb_transport.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,HERE_address,Latitude,Longitude,closest_mrt,dist_mrt_km,closest_cp,dist_cp_km
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4 Singapore,1.37541,103.8374,YIO CHU KANG MRT STATION SINGAPORE,1.0909,BLK 173/176 ANG MO KIO AVE 4,0.1152
1,2015-12,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,69,275000.0,174 ANG MO KIO AVE 4 Singapore,1.37541,103.8374,YIO CHU KANG MRT STATION SINGAPORE,1.0909,BLK 173/176 ANG MO KIO AVE 4,0.1152
2,2016-05,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,68,310000.0,174 ANG MO KIO AVE 4 Singapore,1.37541,103.8374,YIO CHU KANG MRT STATION SINGAPORE,1.0909,BLK 173/176 ANG MO KIO AVE 4,0.1152
3,2016-06,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,68,253000.0,174 ANG MO KIO AVE 4 Singapore,1.37541,103.8374,YIO CHU KANG MRT STATION SINGAPORE,1.0909,BLK 173/176 ANG MO KIO AVE 4,0.1152
4,2016-11,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,68,290000.0,174 ANG MO KIO AVE 4 Singapore,1.37541,103.8374,YIO CHU KANG MRT STATION SINGAPORE,1.0909,BLK 173/176 ANG MO KIO AVE 4,0.1152


In [22]:
amentities = pd.read_csv('./unique_amenities_counts.csv',index_col=0)

In [23]:
amentities.head()

,HERE_address,Latitude,Longitude,schs_1km,schs_2km,sap_sch_1km,sap_sch_2km,autonomous_sch_1km,autonomous_sch_2km,gifted_sch_1km,gifted_sch_2km,ip_sch_1km,ip_sch_2km,hawker_1km,hawker_2km,supermarket_1km,supermarket_2km
0,174 ANG MO KIO AVE 4 Singapore,1.37541,103.83740,6,14,1,2,1,2,0,0,1,1,5,8,4,10
13,541 ANG MO KIO AVE 10 Singapore,1.37412,103.85576,3,11,0,0,1,1,0,0,0,0,2,10,4,15
24,163 ANG MO KIO AVE 4 Singapore,1.37387,103.83834,6,14,1,2,1,2,0,0,1,1,5,8,5,12
33,446 ANG MO KIO AVE 10 Singapore,1.36794,103.85543,5,14,0,1,1,2,0,1,0,1,4,10,5,15
35,557 ANG MO KIO AVE 10 Singapore,1.37145,103.85785,3,12,0,0,1,1,0,1,0,0,2,7,4,14


In [24]:
hdb_amenities = hdb_transport.merge(amentities.drop(['Latitude','Longitude'],axis=1), how='inner', on='HERE_address')

In [25]:
hdb_amenities['flat_model'].unique()

array(['Improved', 'New Generation', 'Adjoined flat', 'Model A',
       'Standard', 'Simplified', 'Premium Apartment', 'Maisonette',
       'Apartment', 'Model A2', 'Type S1', 'Type S2',
       'Premium Maisonette', 'Improved-Maisonette', 'Terrace', 'DBSS',
       'Model A-Maisonette', 'Multi Generation', '2-room',
       'Premium Apartment Loft'], dtype=object)

In [26]:
hdb_amenities.columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price', 'HERE_address', 'Latitude',
       'Longitude', 'closest_mrt', 'dist_mrt_km', 'closest_cp', 'dist_cp_km',
       'schs_1km', 'schs_2km', 'sap_sch_1km', 'sap_sch_2km',
       'autonomous_sch_1km', 'autonomous_sch_2km', 'gifted_sch_1km',
       'gifted_sch_2km', 'ip_sch_1km', 'ip_sch_2km', 'hawker_1km',
       'hawker_2km', 'supermarket_1km', 'supermarket_2km'],
      dtype='object')

In [27]:
dummies = pd.get_dummies(hdb_amenities[['town','storey_range','flat_model','closest_mrt']])

In [28]:
hdb_amenities.drop(['town','storey_range','flat_model'],axis=1,inplace=True)

In [29]:
hdb_amenities_dummies = pd.concat([hdb_amenities,dummies],axis=1)

In [30]:
#choose Flat features, mrt features, schols within 2km, other amenities within 1km
chosen_df = hdb_amenities_dummies.drop(['block','street_name','lease_commence_date','flat_type','HERE_address',
                                        'closest_mrt','closest_cp','dist_cp_km','Latitude','Longitude',
                                        'schs_1km','sap_sch_1km','autonomous_sch_1km','gifted_sch_1km','ip_sch_1km',
                                        'hawker_2km','supermarket_2km'],axis=1).copy()

In [31]:
#Split the data for train and test 
tree_train = chosen_df[~chosen_df['month'].str.contains('2018')].drop(['resale_price','month'],axis=1).copy()
y_train = chosen_df[~chosen_df['month'].str.contains('2018')]['resale_price'].copy()
tree_test = chosen_df[chosen_df['month'].str.contains('2018')].drop(['resale_price','month'],axis=1).copy()
y_test = chosen_df[chosen_df['month'].str.contains('2018')]['resale_price'].copy()

In [32]:
list(tree_train.columns)

['floor_area_sqm',
 'remaining_lease',
 'dist_mrt_km',
 'schs_2km',
 'sap_sch_2km',
 'autonomous_sch_2km',
 'gifted_sch_2km',
 'ip_sch_2km',
 'hawker_1km',
 'supermarket_1km',
 'town_ANG MO KIO',
 'town_BEDOK',
 'town_BISHAN',
 'town_BUKIT BATOK',
 'town_BUKIT MERAH',
 'town_BUKIT PANJANG',
 'town_BUKIT TIMAH',
 'town_CENTRAL AREA',
 'town_CHOA CHU KANG',
 'town_CLEMENTI',
 'town_GEYLANG',
 'town_HOUGANG',
 'town_JURONG EAST',
 'town_JURONG WEST',
 'town_KALLANG/WHAMPOA',
 'town_MARINE PARADE',
 'town_PASIR RIS',
 'town_PUNGGOL',
 'town_QUEENSTOWN',
 'town_SEMBAWANG',
 'town_SENGKANG',
 'town_SERANGOON',
 'town_TAMPINES',
 'town_TOA PAYOH',
 'town_WOODLANDS',
 'town_YISHUN',
 'storey_range_01 TO 03',
 'storey_range_04 TO 06',
 'storey_range_07 TO 09',
 'storey_range_10 TO 12',
 'storey_range_13 TO 15',
 'storey_range_16 TO 18',
 'storey_range_19 TO 21',
 'storey_range_22 TO 24',
 'storey_range_25 TO 27',
 'storey_range_28 TO 30',
 'storey_range_31 TO 33',
 'storey_range_34 TO 36',
 'st

In [33]:
import pickle
final_model = pickle.load(open('./amenities_XGB_model.pickle', 'rb'))

In [34]:
#Run loaded model through same summary test to ensure its reading values right
from sklearn.metrics import mean_squared_error
print('RMSE: ',np.sqrt(mean_squared_error(y_test,final_model.predict(tree_test))))
print('max error:', max(y_test-final_model.predict(tree_test)))
print('max % error: {:.2f}'.format(max(abs(y_test-final_model.predict(tree_test))/y_test)*100))
print('avg % error: {:.2f}'.format((sum((abs(y_test-final_model.predict(tree_test))/y_test))/len(y_test)*100)))

RMSE:  34006.698619912706
max error: 252014.5625
max % error: 107.90
avg % error: 6.25


In [35]:
#Show price with comma separator for thousands
"{:,}".format(int(final_model.predict(tree_train.iloc[[0]])) )

'274,162'

In [36]:
entry = pd.DataFrame(data=np.zeros([1,160]), columns=tree_train.columns.to_list())

In [37]:
entry

,floor_area_sqm,remaining_lease,dist_mrt_km,schs_2km,sap_sch_2km,autonomous_sch_2km,gifted_sch_2km,ip_sch_2km,hawker_1km,supermarket_1km,...,closest_mrt_TELOK BLANGAH MRT STATION SINGAPORE,closest_mrt_TIONG BAHRU MRT STATION SINGAPORE,closest_mrt_TOA PAYOH MRT STATION SINGAPORE,closest_mrt_UBI MRT STATION SINGAPORE,closest_mrt_UPPER CHANGI MRT STATION SINGAPORE,closest_mrt_WOODLANDS MRT STATION SINGAPORE,closest_mrt_WOODLEIGH MRT STATION SINGAPORE,closest_mrt_YEW TEE MRT STATION SINGAPORE,closest_mrt_YIO CHU KANG MRT STATION SINGAPORE,closest_mrt_YISHUN MRT STATION SINGAPORE
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
entry.columns.to_list()

['floor_area_sqm',
 'remaining_lease',
 'dist_mrt_km',
 'schs_2km',
 'sap_sch_2km',
 'autonomous_sch_2km',
 'gifted_sch_2km',
 'ip_sch_2km',
 'hawker_1km',
 'supermarket_1km',
 'town_ANG MO KIO',
 'town_BEDOK',
 'town_BISHAN',
 'town_BUKIT BATOK',
 'town_BUKIT MERAH',
 'town_BUKIT PANJANG',
 'town_BUKIT TIMAH',
 'town_CENTRAL AREA',
 'town_CHOA CHU KANG',
 'town_CLEMENTI',
 'town_GEYLANG',
 'town_HOUGANG',
 'town_JURONG EAST',
 'town_JURONG WEST',
 'town_KALLANG/WHAMPOA',
 'town_MARINE PARADE',
 'town_PASIR RIS',
 'town_PUNGGOL',
 'town_QUEENSTOWN',
 'town_SEMBAWANG',
 'town_SENGKANG',
 'town_SERANGOON',
 'town_TAMPINES',
 'town_TOA PAYOH',
 'town_WOODLANDS',
 'town_YISHUN',
 'storey_range_01 TO 03',
 'storey_range_04 TO 06',
 'storey_range_07 TO 09',
 'storey_range_10 TO 12',
 'storey_range_13 TO 15',
 'storey_range_16 TO 18',
 'storey_range_19 TO 21',
 'storey_range_22 TO 24',
 'storey_range_25 TO 27',
 'storey_range_28 TO 30',
 'storey_range_31 TO 33',
 'storey_range_34 TO 36',
 'st

In [39]:
entry.to_csv('empty_entry.csv')

In [40]:
entry['schs_2km']=amenities_dict["num_sch"]
entry['hawker_1km']=amenities_dict["num_hawkers"]
entry['supermarket_1km']=amenities_dict["num_sm"]
entry['sap_sch_2km']=amenities_dict["num_sap"]
entry['autonomous_sch_2km']=amenities_dict["num_autonomous"]
entry['gifted_sch_2km']=amenities_dict["num_gifted"]
entry['ip_sch_2km']=amenities_dict["num_ip"]

In [41]:
entry

,floor_area_sqm,remaining_lease,dist_mrt_km,schs_2km,sap_sch_2km,autonomous_sch_2km,gifted_sch_2km,ip_sch_2km,hawker_1km,supermarket_1km,...,closest_mrt_TELOK BLANGAH MRT STATION SINGAPORE,closest_mrt_TIONG BAHRU MRT STATION SINGAPORE,closest_mrt_TOA PAYOH MRT STATION SINGAPORE,closest_mrt_UBI MRT STATION SINGAPORE,closest_mrt_UPPER CHANGI MRT STATION SINGAPORE,closest_mrt_WOODLANDS MRT STATION SINGAPORE,closest_mrt_WOODLEIGH MRT STATION SINGAPORE,closest_mrt_YEW TEE MRT STATION SINGAPORE,closest_mrt_YIO CHU KANG MRT STATION SINGAPORE,closest_mrt_YISHUN MRT STATION SINGAPORE
0,0.0,0.0,0.0,6,0,0,0,0,2,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
1%2, 5//2

(1, 2)

In [43]:
storey_select = pd.DataFrame(data = np.arange(0,17),
                             index=['storey_range_01 TO 03',
                                     'storey_range_04 TO 06',
                                     'storey_range_07 TO 09',
                                     'storey_range_10 TO 12',
                                     'storey_range_13 TO 15',
                                     'storey_range_16 TO 18',
                                     'storey_range_19 TO 21',
                                     'storey_range_22 TO 24',
                                     'storey_range_25 TO 27',
                                     'storey_range_28 TO 30',
                                     'storey_range_31 TO 33',
                                     'storey_range_34 TO 36',
                                     'storey_range_37 TO 39',
                                     'storey_range_40 TO 42',
                                     'storey_range_43 TO 45',
                                     'storey_range_46 TO 48',
                                     'storey_range_49 TO 51',],
                                    columns=['Quotient'])

In [44]:
storey_select

,Quotient
storey_range_01 TO 03,0
storey_range_04 TO 06,1
storey_range_07 TO 09,2
storey_range_10 TO 12,3
storey_range_13 TO 15,4
storey_range_16 TO 18,5
storey_range_19 TO 21,6
storey_range_22 TO 24,7
storey_range_25 TO 27,8
storey_range_28 TO 30,9


In [45]:
storey_select.to_csv('storey_select.csv')

In [46]:
test = pd.read_csv('./storey_select.csv',index_col=0)

In [47]:
test

,Quotient
storey_range_01 TO 03,0
storey_range_04 TO 06,1
storey_range_07 TO 09,2
storey_range_10 TO 12,3
storey_range_13 TO 15,4
storey_range_16 TO 18,5
storey_range_19 TO 21,6
storey_range_22 TO 24,7
storey_range_25 TO 27,8
storey_range_28 TO 30,9


In [48]:
storey = 51

if storey > 51:
    storey = 51

if storey%3==0:
    storey-=1
    
storey_select[storey_select['Quotient']==(storey//3)]

,Quotient
storey_range_49 TO 51,16


In [49]:
entry[storey_select[storey_select['Quotient']==(storey//3)].index]

,storey_range_49 TO 51
0,0.0


In [50]:
entry[storey_select[storey_select['Quotient']==(storey//3)].index]=1
entry[storey_select[storey_select['Quotient']==(storey//3)].index]

,storey_range_49 TO 51
0,1.0


In [51]:
#Retrieve nearby houses sold

In [52]:
house_latlong = pd.read_csv("./hdb_inc_latlong.csv", index_col=0)

In [53]:
house_latlong.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,HERE_address,Latitude,Longitude
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4 Singapore,1.37541,103.8374
1,2015-12,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,69,275000.0,174 ANG MO KIO AVE 4 Singapore,1.37541,103.8374
2,2016-05,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,68,310000.0,174 ANG MO KIO AVE 4 Singapore,1.37541,103.8374
3,2016-06,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,68,253000.0,174 ANG MO KIO AVE 4 Singapore,1.37541,103.8374
4,2016-11,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,68,290000.0,174 ANG MO KIO AVE 4 Singapore,1.37541,103.8374


In [54]:
#flat latlong
latitude,longitude

(1.27388354188483, 103.839003328569)

In [55]:
geodesic((1.27442754696067, 103.834878784712),
                         (1.27442754696067+0.0045, 103.834878784712)).km

0.4975867215632862

In [56]:
flats_500m = house_latlong[(abs(house_latlong['Latitude']-latitude)<0.0045)&(abs(house_latlong['Longitude']-longitude)<0.0045)]
flats_500m

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,HERE_address,Latitude,Longitude
2894,2015-01,BUKIT MERAH,3 ROOM,11,CANTONMENT CL,01 TO 03,60.0,Model A,2002,87,420000.0,11 CANTONMENT CL Singapore,1.27466,103.83974
2895,2015-01,BUKIT MERAH,3 ROOM,11,CANTONMENT CL,07 TO 09,60.0,Model A,2002,86,490000.0,11 CANTONMENT CL Singapore,1.27466,103.83974
2896,2015-02,BUKIT MERAH,3 ROOM,11,CANTONMENT CL,19 TO 21,60.0,Model A,2002,86,498000.0,11 CANTONMENT CL Singapore,1.27466,103.83974
2897,2015-04,BUKIT MERAH,4 ROOM,11,CANTONMENT CL,07 TO 09,90.0,Model A,2002,86,720000.0,11 CANTONMENT CL Singapore,1.27466,103.83974
2898,2015-07,BUKIT MERAH,3 ROOM,11,CANTONMENT CL,04 TO 06,60.0,Model A,2002,86,450000.0,11 CANTONMENT CL Singapore,1.27466,103.83974
2899,2015-10,BUKIT MERAH,3 ROOM,11,CANTONMENT CL,10 TO 12,60.0,Model A,2002,86,485000.0,11 CANTONMENT CL Singapore,1.27466,103.83974
2900,2016-05,BUKIT MERAH,4 ROOM,11,CANTONMENT CL,13 TO 15,75.0,Model A,2002,85,632000.0,11 CANTONMENT CL Singapore,1.27466,103.83974
2901,2016-05,BUKIT MERAH,5 ROOM,11,CANTONMENT CL,04 TO 06,110.0,Improved,2002,85,820000.0,11 CANTONMENT CL Singapore,1.27466,103.83974
2902,2016-06,BUKIT MERAH,3 ROOM,11,CANTONMENT CL,13 TO 15,60.0,Model A,2002,85,460000.0,11 CANTONMENT CL Singapore,1.27466,103.83974
2903,2018-01,BUKIT MERAH,3 ROOM,11,CANTONMENT CL,25 TO 27,60.0,Model A,2002,83,520000.0,11 CANTONMENT CL Singapore,1.27466,103.83974


In [57]:
flats_500m.index.shape

(601,)

In [58]:
drop_flats  = []
for index in flats_500m.index:
    if geodesic(flat,(flats_500m.loc[index]['Latitude'],flats_500m.loc[index]['Longitude'])).km < 0.5:
        drop_flats.append(index)

latest_sales_500m = flats_500m.drop(drop_flats,axis=0).iloc[:,:-3].sort_values('month',ascending=False)

In [59]:
latest_sales_500m.columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price'],
      dtype='object')